In [262]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import requests
import openpyxl
import urllib
from urllib import parse
import lxml
from openpyxl import Workbook
from bs4 import BeautifulSoup
from openpyxl.utils.dataframe import dataframe_to_rows
from tensorflow.keras import layers
from nltk.corpus import stopwords
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [263]:
def StockNormIdx(code):
    get_param = {
        'pGB':1,
        'gicode':'A%s'%(code),
        'cID':'',
        'MenuYn':'Y',
        'ReportGB':'',
        'NewMenuID':101,
        'stkGb':701,
    }
    get_param = parse.urlencode(get_param)
    url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
    tables = pd.read_html(url, header=0,encoding='utf-8')
    sit=np.array(tables[11]) 
    print("ROA, ROE, EPS, BPS, DPS, PER, PBR")
    data=[]
    ROE=np.array([])
    for i in range(7):
        ROE = sit[17+i][2:8].astype('float32')
        data.append(ROE/np.linalg.norm(ROE, ord=1))
    data=np.array(data)

    return data

# 주가 예측 

In [272]:
data = StockNormIdx('005930')
print(data)
print('--------------')
print(data[:,0])

ROA, ROE, EPS, BPS, DPS, PER, PBR
[[0.27337417 0.1241352  0.14291361 0.19608617 0.16445936 0.09903143]
 [0.27757356 0.12287896 0.14126132 0.1968326  0.16430996 0.09714367]
 [0.21853007 0.11485163 0.13933831 0.20956977 0.19415222 0.12355801]
 [0.13861348 0.1471871  0.15455273 0.17104499 0.19023564 0.19836608]
 [0.13715614 0.13715614 0.29000387 0.13986827 0.14732662 0.14848895]
 [0.07525495 0.20665807 0.24721605 0.1588325  0.12132224 0.19071622]
 [0.1265823  0.17146146 0.2370541  0.20713465 0.13118528 0.1265823 ]]
--------------
[0.27337417 0.27757356 0.21853007 0.13861348 0.13715614 0.07525495
 0.1265823 ]


In [298]:
samsung = pd.read_csv('./data_in/samsung.csv')
sam = samsung[:][:1148]
sa = np.array(sam)
sa = sa[:,2].reshape(-1,1)
print(sa.shape)
sa2023 = sa[:-1147]
sa2022 = sa[-1147:-901]
sa2021 = sa[-901:-653]
sa2020 = sa[-653:-405]
sa2019 = sa[-405:-159]
sa2018 = sa[-159:len(sa)]

(1148, 1)


C:\Users\wjddu\AppData\Local\Temp\ipykernel_10064\1246311570.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  samsung = pd.read_csv('./data_in/samsung.csv')


In [299]:
data2023 = np.full((len(sa2023),7), data[:,5])
data2022 = np.full((len(sa2022),7), data[:,4])
data2021 = np.full((len(sa2021),7), data[:,3])
data2020 = np.full((len(sa2020),7), data[:,2])
data2019 = np.full((len(sa2019),7), data[:,1])
data2018 = np.full((len(sa2018),7), data[:,0])
data1 = data2023
data1 = np.concatenate((data2023,data2022,data2021,data2020,data2019,data2018))
sa = np.concatenate((sa, data1), axis=1)
print(sa.shape)

(1148, 8)
